In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import os
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score,accuracy_score,confusion_matrix,precision_score,f1_score,recall_score
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.ensemble import GradientBoostingClassifier
from nltk.stem import PorterStemmer
import nltk


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


for dirname, _, filenames in os.walk('/kaggle/input'):   #print all the files in the input of kaggle
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
url_train = "https://raw.githubusercontent.com/ezhil384/Youtube-text-classification/master/train.csv"
url_test = "https://raw.githubusercontent.com/ezhil384/Youtube-text-classification/master/test.csv"
download1 = requests.get(url_train).content
download2 = requests.get(url_test).content
train_data = pd.read_csv(io.StringIO(download1.decode('utf-8')))
test_data = pd.read_csv(io.StringIO(download2.decode('utf-8')))
train_data.head()

In [ ]:
train_data = pd.read_csv("../input/youtube/train.csv")
test_data = pd.read_csv("../input/youtube/test.csv")
train_data.head()

In [ ]:
test_data.head()

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(train_data.isnull().values.any())   #check for dropping and pre processing
print(test_data.isnull().values.any())

In [ ]:
# loading stop words from nltk library
ending_words = set(stopwords.words('english'))

def desc_process_train(text, index, column):
    if type(text) is not int:
        s = ""
        url = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?' #removing URL's
        text = re.sub(url,' ',text)
        email =r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+'  #removing mails
        text = re.sub(email,' ', text)
        text = re.sub('[^a-zA-Z0-9\n]',' ',text)    #removing all special characters
        text = re.sub('\s+',' ',text)    #remove multi spaces
        text = text.lower()
        
        for word in text.split():
            if not word in ending_words:
                s = s + word + " "
        
        train_data[column][index] = s
for ind, row in train_data.iterrows():
    if type(row['description']) is str:
        desc_process_train(row['description'], ind, 'description')

In [ ]:
train_data.head()

In [ ]:
categories=train_data['category_id'].unique()
print(categories)

# Splitting the data into train test and validate

In [ ]:
y = train_data['category_id'].values
# split the data into test and train by maintaining same distribution of output varaible 'y_true' [stratify=y_true]
train, test, y_train, y_test = train_test_split(train_data, y, stratify=y, test_size=0.2)
print('Data points in training data:', train.shape[0])
print('Data points in test data:', test.shape[0])

# Graph to show the distribution of the data into all 16 categories

In [ ]:
X_category=train['category_id'].value_counts()   #values in each category in X
plt.figure(figsize=(12,12))
sns.barplot(X_category.index, X_category.values, alpha=0.8)
plt.title('Distribution of the categories in the training data set')
plt.ylabel('No. of data')
plt.xlabel('Category')
plt.show()

# Using tf_idf to vectorize the words present in the description

In [ ]:
x_train=train['description']
x_test=test['description']
vector_texts = TfidfVectorizer(ngram_range=(1,2))
x_tr_tfidf = vector_texts.fit_transform(x_train)
x_test_tfidf = vector_texts.transform(x_test)

# Using Stochastic gradient descent classifier for a linear SVM to train the data and predict the results

In [ ]:
clf = SGDClassifier(loss = 'hinge', alpha =0.00005, penalty='l2', class_weight='balanced', learning_rate='optimal',eta0=0.001, n_jobs = -1) 
clf.fit(x_tr_tfidf,y_train)
y_pred = clf.predict(x_test_tfidf)
y2_pred = clf.predict(x_tr_tfidf)
print("Accuracy on train set: %0.3f%%"%(accuracy_score(y_train, y2_pred)*100))
print("Accuracy on test set: %0.3f%%"%(accuracy_score(y_test, y_pred)*100))
print("Precision on test set: %0.3f"%(precision_score(y_test, y_pred,average='macro')))
print("Recall on test set: %0.3f"%(recall_score(y_test, y_pred,average='macro')))
print("F1-Score on test set: %0.3f"%(f1_score(y_test, y_pred,average='macro')))

print("-"*20, "confusion matrix", "-"*20)
plt.figure(figsize=(12,8))
df_cm = pd.DataFrame(confusion_matrix(y_test, y_pred), range(15),range(15))
sns.set(font_scale=1.4)#for label size
labels = categories
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, fmt='g',xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted categories')
plt.ylabel('Original categories')
plt.show()

# Using the API for SVM

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clvmf = SVC(kernel='linear',C=1.5,gamma='scale') 
clvmf.fit(x_tr_tfidf,y_train)
y_predf = clvmf.predict(x_test_tfidf)
y2_predf = clvmf.predict(x_tr_tfidf)
print("Accuracy on train set: %0.3f%%"%(accuracy_score(y_train, y2_predf)*100))
print("Accuracy on test set: %0.3f%%"%(accuracy_score(y_test, y_predf)*100))
print("Precision on test set: %0.3f"%(precision_score(y_test, y_predf,average='macro')))
print("Recall on test set: %0.3f"%(recall_score(y_test, y_predf,average='macro')))
print("F1-Score on test set: %0.3f"%(f1_score(y_test, y_predf,average='macro')))

print("-"*20, "confusion matrix", "-"*20)
plt.figure(figsize=(12,8))
df_cm = pd.DataFrame(confusion_matrix(y_test, y_predf), range(15),range(15))
sns.set(font_scale=1.4)#for label size
labels = categories
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, fmt='g',xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Class')
plt.ylabel('Original Class')
plt.show()

# Using XGBoost algorithm to check as an alternative if it performs well

In [ ]:
xgb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.01, max_depth=3, random_state=0)
xgb.fit(x_tr_tfidf,y_train)
y_pred = xgb.predict(x_test_tfidf)
print("Accuracy on test set: %0.3f%%"%(accuracy_score(y_test, y_pred)*100))
print("Precision on test set: %0.3f"%(precision_score(y_test, y_pred,average='macro')))
print("Recall on test set: %0.3f"%(recall_score(y_test, y_pred,average='macro')))
print("F1-Score on test set: %0.3f"%(f1_score(y_test, y_pred,average='macro')))

print("-"*20, "confusion matrix", "-"*20)
plt.figure(figsize=(12,8))
df_cm = pd.DataFrame(confusion_matrix(y_test, y_pred), range(15),range(15))
sns.set(font_scale=1.4)#for label size
labels = categories
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, fmt='g',xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Class')
plt.ylabel('Original Class')
plt.show()

# The test file data

In [ ]:
def desc_process_test(text, index, column):
    if type(text) is not int:
        s = ""
        url = r'((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?' #removing URL's
        text = re.sub(url,' ',text)
        email =r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+'  #removing mails
        text = re.sub(email,' ', text)
        text = re.sub('[^a-zA-Z0-9\n]',' ',text)    #removing all special characters
        text = re.sub('\s+',' ',text)    #remove multi spaces
        text = text.lower()
        
        for word in text.split():
            if not word in ending_words:
                s = s + word + " "
        
        test_data[column][index] = s

**Pre processing the description of the test dataset**

In [ ]:
for index, row in test_data.iterrows():
    if type(row['description']) is str:
        desc_process_test(row['description'], index, 'description')

In [ ]:
test_data.head()

# Vectorizing the test_dataset description word counts and then running the SVM classifier

In [ ]:
x_testdata = vector_texts.transform(test_data['description'])
y_testpred = clvmf.predict(x_testdata)
print(y_testpred)

In [ ]:
y_testpred = clf.predict(x_testdata)
print(y_testpred)

# Creating the final csv file with the predictions

In [ ]:
import csv
met = "../input/youtube/test.csv"
with open('submission.csv','w') as fappend:
    csvwriter=csv.writer(fappend)
    csvwriter.writerow(['video_id','category_id'])
with open(met, 'r') as fread,open('submission.csv','a') as fappend:
    csvreader = csv.reader(fread)
    csvwriter = csv.writer(fappend)
    i=0
    fr=0   #row 0 check to avoid first row with heading
    for row in csvreader:
        if fr==0:
            fr=1
            continue
        csvwriter.writerow([row[0],y_testpred[i]])
        i=i+1
    
          

# By this we conclude that linear SVM is the best model for text classification for the 16 categories.